# Problem-- 1)

Use Random Forest to prepare a model on fraud data treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

#### Data Description :

Undergrad : person is under graduated or not

Marital.Status : marital status of a person

Taxable.Income : Taxable income is the amount of how much tax an individual owes to the government

Work Experience : Work experience of an individual person

Urban : Whether that person belongs to urban area or not

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn import preprocessing

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
fraud= pd.read_csv('C:/Users/ancha/Downloads/Fraud_check (1).csv')
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


# EDA

In [3]:
fraud.columns

Index(['Undergrad', 'Marital.Status', 'Taxable.Income', 'City.Population',
       'Work.Experience', 'Urban'],
      dtype='object')

In [4]:
fraud['tax_type']= np.where(fraud['Taxable.Income']<= 30000, 'Risky', 'Good')
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,tax_type
0,NO,Single,68833,50047,10,YES,Good
1,YES,Divorced,33700,134075,18,YES,Good
2,NO,Married,36925,160205,30,YES,Good
3,YES,Single,50190,193264,15,YES,Good
4,NO,Married,81002,27533,28,NO,Good
...,...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES,Good
596,YES,Divorced,69967,55369,2,YES,Good
597,NO,Divorced,47334,154058,0,YES,Good
598,YES,Married,98592,180083,17,NO,Good


In [11]:
fraud.rename(columns={'Undergrad':'undergraduate', 'Marital.Status':'marital_status', 'Taxable.Income':'taxable_income', 'City.Population':'city_population', 'Work.Experience':'work_exp', 'Urban':'urban'},inplace=True)


In [12]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   undergraduate    600 non-null    object
 1   marital_status   600 non-null    object
 2   taxable_income   600 non-null    int64 
 3   city_population  600 non-null    int64 
 4   work_exp         600 non-null    int64 
 5   urban            600 non-null    object
 6   tax_type         600 non-null    object
dtypes: int64(3), object(4)
memory usage: 32.9+ KB


In [13]:
fraud.describe()

,taxable_income,city_population,work_exp
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [14]:
fraud.duplicated().value_counts()

False    600
dtype: int64

In [15]:
for i, column in enumerate(fraud.columns, 1):
    print(fraud[column].value_counts())

YES    312
NO     288
Name: undergraduate, dtype: int64
Single      217
Married     194
Divorced    189
Name: marital_status, dtype: int64
59689    2
68833    1
60822    1
93090    1
79373    1
        ..
61906    1
31088    1
70365    1
56536    1
96519    1
Name: taxable_income, Length: 599, dtype: int64
57194     2
162777    2
50047     1
130912    1
64070     1
         ..
145744    1
58225     1
28661     1
117260    1
158137    1
Name: city_population, Length: 598, dtype: int64
10    31
13    28
30    25
28    25
12    25
14    24
11    23
21    23
5     22
27    22
26    22
7     21
29    21
19    21
16    19
6     19
25    19
2     19
17    18
15    18
24    17
23    16
20    15
0     14
1     14
8     14
3     14
4     14
22    13
9     13
18    11
Name: work_exp, dtype: int64
YES    302
NO     298
Name: urban, dtype: int64
Good     476
Risky    124
Name: tax_type, dtype: int64


# Label Encoding

In [17]:
fraud_lab= fraud.copy()

for i, column in enumerate(fraud_lab[[var for var in fraud_lab.columns if fraud_lab[var].dtype== 'O']].columns, 1):
    
    label_encoder= preprocessing.LabelEncoder()
    fraud_lab[column]= label_encoder.fit_transform(fraud_lab[column])

In [19]:
fraud_lab= fraud_lab.drop('taxable_income',axis=1)

ss= StandardScaler()
ss.fit_transform(fraud_lab)

array([[-1.040833  ,  1.16079994, -1.17852072, -0.62914254,  0.99335541,
        -0.51039612],
       [ 0.96076892, -1.27444469,  0.50850002,  0.27636996,  0.99335541,
        -0.51039612],
       [-1.040833  , -0.05682237,  1.03310911,  1.63463872,  0.99335541,
        -0.51039612],
       ...,
       [-1.040833  , -1.27444469,  0.90969648, -1.76103318,  0.99335541,
        -0.51039612],
       [ 0.96076892, -0.05682237,  1.43219749,  0.1631809 , -1.00668904,
        -0.51039612],
       [-1.040833  , -1.27444469,  0.9915901 ,  0.04999184, -1.00668904,
        -0.51039612]])

# Data Splitting

In [20]:
x= fraud_lab.drop('tax_type',axis=1)
y= fraud_lab['tax_type']

In [21]:
x.shape, y.shape

((600, 5), (600,))

In [22]:
y.value_counts()

0    476
1    124
Name: tax_type, dtype: int64

In [23]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.30, random_state=40)

In [24]:
x

,undergraduate,marital_status,city_population,work_exp,urban
0,0,2,50047,10,1
1,1,0,134075,18,1
2,0,1,160205,30,1
3,1,2,193264,15,1
4,0,1,27533,28,0
...,...,...,...,...,...
595,1,0,39492,7,1
596,1,0,55369,2,1
597,0,0,154058,0,1
598,1,1,180083,17,0


In [26]:
y

0      0
1      0
2      0
3      0
4      0
      ..
595    0
596    0
597    0
598    0
599    0
Name: tax_type, Length: 600, dtype: int32

# Parameters for Random Forest Classifier

In [27]:
def print_score(clf, x_train, y_train, x_test, y_test):
        pred = clf.predict(x_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        
        print(f"\n Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print(f"\n CLASSIFICATION REPORT:\n{clf_report}")
        print(f"\nConfusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

In [36]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix,accuracy_score, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [29]:
params = {'n_estimators': [100, 300, 500],'max_features': ['auto', 'sqrt', 'log2'],"criterion":["gini", "entropy"], "max_depth":(list(range(1,15)))}

rf_clf = RandomForestClassifier(random_state= 40)
tree_cv = GridSearchCV(rf_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=5)
tree_cv.fit(x_train, y_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(x_train, y_train)
print_score(rf_clf, x_train, y_train, x_test, y_test)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits
Best paramters: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'auto', 'n_estimators': 100})

 Accuracy Score: 80.00%

 CLASSIFICATION REPORT:
                    0     1  accuracy   macro avg  weighted avg
precision    0.800000   0.0       0.8    0.400000      0.640000
recall       1.000000   0.0       0.8    0.500000      0.800000
f1-score     0.888889   0.0       0.8    0.444444      0.711111
support    336.000000  84.0       0.8  420.000000    420.000000

Confusion Matrix: 
 [[336   0]
 [ 84   0]]



## Random Forest Classifier -- Entropy

In [31]:
rfclf_ent= RandomForestClassifier(criterion='entropy', max_depth=1, random_state=40, max_features='auto', n_estimators= 100)
rfclf_ent.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=1, random_state=40)

In [32]:
pred= rfclf_ent.predict(x_test)
accuracy_score(y_test, pred)

0.7777777777777778

## Random Forest Classifier -- Gini

In [33]:
rfclf_gini= RandomForestClassifier(criterion='gini', max_depth=1, random_state=40, max_features='auto', n_estimators= 100)
rfclf_gini.fit(x_train, y_train)

RandomForestClassifier(max_depth=1, random_state=40)

In [34]:
pred= rfclf_gini.predict(x_test)
accuracy_score(y_test, pred)

0.7777777777777778

# Bagging

In [37]:
from sklearn.ensemble import BaggingClassifier


kfold= KFold(n_splits=10)
cart= DecisionTreeClassifier()
num_trees= 100

model= BaggingClassifier(base_estimator= cart, n_estimators=100, random_state=40)
results= cross_val_score(model, x, y, cv=kfold)

In [39]:
model.fit(x_train, y_train);
print('\n Confusion Matrix:\n', confusion_matrix(y_test, model.predict(x_test)))
print('\n Accuracy:\n', accuracy_score(y_test, model.predict(x_test)))


 Confusion Matrix:
 [[134   6]
 [ 39   1]]

 Accuracy:
 0.75


# Boosting

In [41]:
#AdaBoost Classification
from sklearn.ensemble import AdaBoostClassifier


num_trees= 10
seed= 0
kfold= KFold(n_splits= 10)

model=AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results= cross_val_score(model, x, y, cv=kfold)

In [42]:
model.fit(x_train, y_train);
print('\n Confusion Matrix:\n', confusion_matrix(y_test, model.predict(x_test)))
print('\n Accuracy:\n', accuracy_score(y_test, model.predict(x_test)))


 Confusion Matrix:
 [[139   1]
 [ 40   0]]

 Accuracy:
 0.7722222222222223


# Conclusion:

In [94]:
# Accuracy of models

Accuracy_tab= {'Model':pd.Series(['rfclf-Gini', 'Bagging', 'Boosting']),'Accuracy':pd.Series(['78%', '75%', '77%'])}
Accuracy_tab= pd.DataFrame(Accuracy_tab)
Accuracy_tab

,Model,Accuracy
0,rfclf-Gini,78%
1,Bagging,75%
2,Boosting,77%


# Problem -- 2)

A cloth manufacturing company is interested to know about the segment or attributes causes high sale.

Approach - A decision tree can be built with target variable Sale (we will first convert it in categorical variable) & all other variable will be independent in the analysis.

About the data:

Let’s consider a Company dataset with around 10 variables and 400 records. The attributes are as follows:

 Sales -- Unit sales (in thousands) at each location

 Competitor Price -- Price charged by competitor at each location

 Income -- Community income level (in thousands of dollars)

 Advertising -- Local advertising budget for company at each location (in thousands of dollars)

 Population -- Population size in region (in thousands)

 Price -- Price company charges for car seats at each site

 Shelf Location at stores -- A factor with levels Bad, Good and Medium indicating the quality of the shelving location for the car seats at each site

 Age -- Average age of the local population

 Education -- Education level at each location

 Urban -- A factor with levels No and Yes to indicate whether the store is in an urban or rural location

 US -- A factor with levels No and Yes to indicate whether the store is in the US or not

In [47]:
company= pd.read_csv('C:/Users/ancha/Downloads/Company_Data (1).csv')
company

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


# EDA

In [52]:
#adding sales category feature
company['Sales_cat']= np.where(company['Sales']<=company['Sales'].mean(), 'High', 'Low')
company.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sales_cat
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,Low
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,Low
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,Low
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,High
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,High


In [54]:
company.Sales_cat= company.Sales_cat.astype(str)

In [55]:
company

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sales_cat
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,Low
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,Low
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,Low
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,High
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,High
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes,Low
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes,High
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes,High
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes,High


In [56]:
company.Sales_cat.value_counts()

High    201
Low     199
Name: Sales_cat, dtype: int64

In [57]:
company.shape

(400, 12)

In [58]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
 11  Sales_cat    400 non-null    object 
dtypes: float64(1), int64(7), object(4)
memory usage: 37.6+ KB


In [59]:
company.duplicated().value_counts()

False    400
dtype: int64

In [60]:
company.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


# Label Encoding

In [61]:
company_lab= company.copy()

for i, column in enumerate(company_lab[[var for var in company_lab.columns if company_lab[var].dtype == 'O']].columns,1):
    label_encoder= preprocessing.LabelEncoder()
    company_lab[column]= label_encoder.fit_transform(company_lab[column])
    
company_lab= company_lab.drop('Sales', axis=1)

In [62]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss= MinMaxScaler()
ss.fit_transform(company_lab)

array([[0.62244898, 0.52525253, 0.37931034, ..., 1.        , 1.        ,
        1.        ],
       [0.34693878, 0.27272727, 0.55172414, ..., 1.        , 1.        ,
        1.        ],
       [0.36734694, 0.14141414, 0.34482759, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.86734694, 0.05050505, 0.4137931 , ..., 1.        , 1.        ,
        0.        ],
       [0.23469388, 0.58585859, 0.24137931, ..., 1.        , 1.        ,
        0.        ],
       [0.58163265, 0.16161616, 0.        , ..., 1.        , 1.        ,
        1.        ]])

# Data Splitting

In [63]:
x= company_lab.drop('Sales_cat', axis=1)
y= company_lab['Sales_cat']

In [64]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.30, random_state=40)

In [65]:
x

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,138,73,11,276,120,0,42,17,1,1
1,111,48,16,260,83,1,65,10,1,1
2,113,35,10,269,80,2,59,12,1,1
3,117,100,4,466,97,2,55,14,1,1
4,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,1,33,14,1,1
396,139,23,3,37,120,2,55,11,0,1
397,162,26,12,368,159,2,40,18,1,1
398,100,79,7,284,95,0,50,12,1,1


In [66]:
y

0      1
1      1
2      1
3      0
4      0
      ..
395    1
396    0
397    0
398    0
399    1
Name: Sales_cat, Length: 400, dtype: int32

# Parameters for Random Forest Classifier

In [78]:
def print_score(clf, x_train, y_train, x_test, y_test):
        pred = clf.predict(x_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print(f"\n Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print(f"\n CLASSIFICATION REPORT:\n{clf_report}")
        print(f"\nConfusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

In [79]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix,accuracy_score, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [80]:
params = {'n_estimators': [100,300,500],'max_features': ['auto', 'sqrt', 'log2'],"criterion":["gini", "entropy"], "max_depth":(list(range(1,15)))}

rf_clf = RandomForestClassifier(random_state= 40)
tree_cv = GridSearchCV(rf_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=5)
tree_cv.fit(x_train, y_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(x_train, y_train)
print_score(rf_clf, x_train, y_train, x_test, y_test)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits
Best paramters: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 100})

 Accuracy Score: 94.64%

 CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.975806    0.923077  0.946429    0.949442      0.948123
recall       0.909774    0.979592  0.946429    0.944683      0.946429
f1-score     0.941634    0.950495  0.946429    0.946065      0.946286
support    133.000000  147.000000  0.946429  280.000000    280.000000

Confusion Matrix: 
 [[121  12]
 [  3 144]]



### Random Forest Classifier -- Entropy

In [81]:
rfclf_entropy=RandomForestClassifier(criterion='entropy',max_depth=8,random_state= 40,max_features= 'auto', n_estimators= 100)
rfclf_entropy.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=8, random_state=40)

In [82]:
pred=rfclf_entropy.predict(x_test)
accuracy_score(y_test,pred)

0.75

### Random Forest Classifier -- Gini

In [83]:
rfclf_gini=RandomForestClassifier(criterion='gini',max_depth=8,random_state= 40,max_features= 'auto', n_estimators= 100)
rfclf_gini.fit(x_train,y_train)

RandomForestClassifier(max_depth=8, random_state=40)

In [84]:
pred=rfclf_gini.predict(x_test)
accuracy_score(y_test,pred)

0.75

# Bagging

In [87]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

kfold=KFold(n_splits=10)
cart=DecisionTreeClassifier()
num_trees = 100

model=BaggingClassifier(base_estimator=cart,n_estimators=100,random_state=40)
results=cross_val_score(model, x, y, cv=kfold)
     

In [88]:
model.fit(x_train, y_train);
print('\n confusion matrix :\n',confusion_matrix(y_test,model.predict(x_test)))
print('\n accuarcy :\n',accuracy_score(y_test,model.predict(x_test)))


 confusion matrix :
 [[52 16]
 [13 39]]

 accuarcy :
 0.7583333333333333


# Boosting

In [90]:
from pandas.core.common import random_state
from pandas import read_csv
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import AdaBoostClassifier

num_trees = 10
seed=0
kfold = KFold(n_splits=10)

model=AdaBoostClassifier(n_estimators=num_trees,random_state=seed)

In [91]:
model.fit(x_train, y_train);
print('\n confusion matrix :\n',confusion_matrix(y_test,model.predict(x_test)))
print('\n accuarcy :\n',accuracy_score(y_test,model.predict(x_test)))


 confusion matrix :
 [[50 18]
 [11 41]]

 accuarcy :
 0.7583333333333333


# Conclusion

In [93]:
# comparing accuracy of models
Accuracy_tab={'Model':pd.Series(['rfclf_Entropy', 'rfclf_Gini', 'Bagging', 'Boosting']),
     'accuracy':pd.Series(['75%','75%','76%','76%'])}
     
Accuracy_tab=pd.DataFrame(Accuracy_tab)
Accuracy_tab

,Model,accuracy
0,rfclf_Entropy,75%
1,rfclf_Gini,75%
2,Bagging,76%
3,Boosting,76%
